In [1]:
from task import CreateDataBeforeBatch,TMPDataset,CreateLable,MapAtomNode,node_accuracy,GaussianSmoothing,batchdata
from data_utils import ProcessRawData,ParseStructure
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from egnnmodel import EGNNModel
import numpy as np
from scipy.ndimage import gaussian_filter1d
from test import TMPTest
import warnings
warnings.filterwarnings('ignore')

[12/19/23 14:19:06] WARNING  To use the Graphein submodule                                         ]8;id=607912;file:///zhome/77/2/193848/.local/lib/python3.9/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=437758;file:///zhome/77/2/193848/.local/lib/python3.9/site-packages/graphein/protein/features/sequence/embeddings.py#44\44]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             To do so, use the following command: pip install biovec                               
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

[12/19/23 14:19:07] WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=940322;file:///zhome/77/2/193848/.local/lib/python3.9/site-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=944719;file:///zhome/77/2/193848/.local/lib/python3.9/site-packages/graphein/protein/visualisation.py#35\35]8;;\
                             need to install: pytorch3d                                                            
                             pytorch3d cannot be installed via pip                                                 

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=822435;file:///zhome/77/2/193848/.local/lib/python3.9/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=385367;file:///zhome/77/2/193848/.local/lib/python3.9/site-packages/graphein/protein/meshes.py#29\29]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: pip install pytorch3d                            

In [2]:
import pickle
path = '/work3/s230027/DL/codebase/dataset/parse raw data/cv4.pickle'
with open(path, 'rb') as file:
    cv4 = pickle.load(file)

In [3]:

TM_name_list = cv4[cv4['protein_type'] == 'TM']['uniprot_id_low'].tolist()
BETA_name_list = cv4[cv4['protein_type'] == 'BETA']['uniprot_id_low'].tolist()
SP_TM_name_list = cv4[cv4['protein_type'] == 'SP+TM']['uniprot_id_low'].tolist()


In [4]:
file_name = "DeepTMHMM.3line"
path='/work3/s230027/DL/codebase/'
batch_size=100
setup = 'setup1' # choose crossvalidation (total 5)
processsor= CreateDataBeforeBatch(path)
train_data_dict_before_batch,val_data_dict_before_batch,test_data_dict_before_batch=processsor.get_data(setup)

TM_test={}
for name in TM_name_list:
    TM_test[name]=test_data_dict_before_batch[name]

BETA_test={}
for name in BETA_name_list:
    BETA_test[name]=test_data_dict_before_batch[name]

SP_TM_test={}
for name in SP_TM_name_list:
    SP_TM_test[name] = test_data_dict_before_batch[name]


In [5]:

file_name = "DeepTMHMM.3line"
path='/work3/s230027/DL/codebase/'
modelpath = '/work3/s230027/DL/result/egnn/egnn_model_size1_epoch100.pth'
batch_size=1

In [6]:
##TM
processor=TMPTest(TM_test,file_name,path,batch_size,5,setup='setup1',modelpath=modelpath)
processor.printresult()

Node acc: 0.3825377447296664
Node binary acc: 0.4297083089716105
Avg atom acc: 0.38577157575189813
Total atom acc: 1360.4285714285713
Correct topology without Gaussian smoothing: 0.0
Node acc before smoothing: 0.3825377447296664
Before smoothing correct topology: 0.0 

Node acc after Gaussian smoothing: 0.38276178633662916
After Gaussian smoothing correct topology: 0.0 



In [7]:
##SP_TM
processor=TMPTest(SP_TM_test,file_name,path,batch_size,5,setup='setup1',modelpath=modelpath)
processor.printresult()

Node acc: 0.21994157645853893
Node binary acc: 0.23343890208498286
Avg atom acc: 0.22268864820280432
Total atom acc: 1416.7
Correct topology without Gaussian smoothing: 0.0
Node acc before smoothing: 0.21994157645853893
Before smoothing correct topology: 0.0 

Node acc after Gaussian smoothing: 0.22011942967590584
After Gaussian smoothing correct topology: 0.0 



In [8]:
##BETA
processor=TMPTest(BETA_test,file_name,path,batch_size,3,setup='setup1',modelpath=modelpath)
processor.printresult()

Node acc: 0.0006105413463995653
Node binary acc: 0.00031443244942878104
Avg atom acc: 0.0013704833060037974
Total atom acc: 4.125
Correct topology without Gaussian smoothing: 0.0
Node acc before smoothing: 0.0006105413463995653
Before smoothing correct topology: 0.0 

Node acc after Gaussian smoothing: 0.0
After Gaussian smoothing correct topology: 0.0 



In [9]:
# all the test data
processor=TMPTest(test_data_dict_before_batch,file_name,path,batch_size,3,setup='setup1',modelpath=modelpath)
processor.printresult()

Node acc: 0.606905556011921
Node binary acc: 0.6794101413792616
Avg atom acc: 0.6057389078246742
Total atom acc: 2655.2419127988746
Correct topology without Gaussian smoothing: 0.3319268635724332
Node acc before smoothing: 0.606905556011921
Before smoothing correct topology: 0.3319268635724332 

Node acc after Gaussian smoothing: 0.6074191928852865
After Gaussian smoothing correct topology: 0.559774964838256 

